# PENSA Tutorial Using GPCRmd Trajectories
Here we show some common functions included in PENSA, using trajectories of a G protein-coupled receptor (GPCR). We retrieve the molecular dynamics trajectories for this tutorial from [GPCRmd](https://submission.gpcrmd.org/home/), an online platform for collection and curation of GPCR simulations. It is described in more detail [here](https://www.nature.com/articles/s41592-020-0884-y).

<p align="center">
<img src="https://pbs.twimg.com/media/Ej8-VJ5WkAAbgJc?format=jpg&name=large" width="500">
</p>


The example system is the mu-opioid receptor (mOR), once in its apo form and once bound to the ligand [BU72](https://www.guidetopharmacology.org/GRAC/LigandDisplayForward?ligandId=9363). The structure of this GPCR (G protein-coupled receptor) is reported by [*Huang et al (2015)*](https://www.nature.com/articles/nature14886). 
We are going to compare the structural ensembles of the receptor in these two conditions.

This tutorial assumes that you can download the trajectories (see below). If you can't, you can use any other system you have available and adapt the file names and residue selections accordingly.

We only need to import the module "os" and all functions from PENSA itself which in turn loads all the modules it needs.

In [ ]:
import os
from pensa import *

## Download

PENSA has a predefined function to download GPCRmd trajectories.

In [ ]:
# Define where to save the GPCRmd files
root_dir = './mor-data'
# Define which files to download
md_files = ['11427_dyn_151.psf','11426_dyn_151.pdb', # MOR-apo
            '11423_trj_151.xtc','11424_trj_151.xtc','11425_trj_151.xtc',
            '11580_dyn_169.psf','11579_dyn_169.pdb', # MOR-BU72
            '11576_trj_169.xtc','11577_trj_169.xtc','11578_trj_169.xtc']
# Download all the files that do not exist yet
for file in md_files:
    if not os.path.exists(os.path.join(root_dir,file)):
        download_from_gpcrmd(file,root_dir)

## Preprocessing 

To work with the protein coordinates, we first need to extract them from the simulation, i.e., remove the solvent, lipids etc. This is the hardest part but you usually only have to do it once and can then play with your data. Preprocessing can handle many common trajectory formats (as it is based on MDAnalysis) but the internal featurization (based on PyEMMA) is a bit more restrictive, so we will always write xtc trajectories. For large trajectories, you might want to use the scripts provided in the PENSA repository, e.g., to run them on the computing cluster and then download the processed data. Once you know how PENSA works, you can write your own scripts.

In the following, we define the necessary files. For each simulation, we need a reference file (.psf for AMBER), a PDB file, and the trajetory. 

Make sure to adapt the root directory such that it links to wherever you have mounted Oak. I you cannot access the Sherlock cluster at Stanford, use any other simulation that you would like to compare. 

To run this tutorial on another system, you'll have to adapt the file paths and names in the following box and, in case you need them, the residue selections in the folder ```selections```. We explain how they work further below. Note that for some PENSA functions it is sufficient that the derived features are the same while for others (especially those that involve trajectory manipulation), all atoms need to be the same.  In our particular example, we exclude hydrogen atoms because residue Asp114 is protonated in the BU72 simulation but not in the apo simulation.

In [ ]:
root_dir = './mor-data'
# Simulation A
ref_file_a =  root_dir+'/11427_dyn_151.psf'
pdb_file_a =  root_dir+'/11426_dyn_151.pdb'
trj_file_a = [root_dir+'/11423_trj_151.xtc',
              root_dir+'/11424_trj_151.xtc',
              root_dir+'/11425_trj_151.xtc']
# Simulation B
ref_file_b =  root_dir+'/11580_dyn_169.psf'
pdb_file_b =  root_dir+'/11579_dyn_169.pdb'
trj_file_b = [root_dir+'/11576_trj_169.xtc',
              root_dir+'/11577_trj_169.xtc',
              root_dir+'/11578_trj_169.xtc']
# Base for the selection string for each simulation
sel_base_a = "(not name H*) and protein"
sel_base_b = "(not name H*) and protein"
# Names of the output files
out_name_a = "traj/condition-a"
out_name_b = "traj/condition-b"
out_name_combined="traj/combined"

For this tutorial, we will save the processed trajectories in the subfolder ```traj```. We also create subfolders for other results that we will generate.

In [ ]:
for subdir in ['traj','plots','vispdb','pca','clusters','results']:
    if not os.path.exists(subdir):
        os.makedirs(subdir)

We have to ensure that from both simulations, we use the exact same parts of the receptor for the analysis. Often, this will be easy and you just provide a simple selection string for the corresponding segment. For more complicated cases, we can use the function ```load_selection()``` to generate a complete residue list from a plain text file. This file should provide in each line the first and the last residue to be considered for a part of the protein. 

In the first case, we will extract all protein residues, assuming (correctly) that the same ones are present in both simulations.

In [ ]:
# Extract the coordinates of the receptor from the trajectory
extract_coordinates(ref_file_a, pdb_file_a, trj_file_a, out_name_a+"_receptor", sel_base_a)
extract_coordinates(ref_file_b, pdb_file_b, trj_file_b, out_name_b+"_receptor", sel_base_b)

In many cases, you probably have several runs of the same simulation that you want to combine to one structural ensemble. This is why the trajectory argument takes a list as arguments, e.g.

    extract_coordinates(system.psf, system.pdb, ['run1.nc','run2.nc','run3.nc'], 
                        'rho_receptor', 'protein', start_frame=1000)
                                 
With the option ```start_frame```, you can exclude the equilibrium phase already at this stage. Be aware that in combined simulations, there is no straightforward way to exclude it later as it would require bookkeeping about how long each simulation was etc.

For some analysis types, we only want to use the part of the receptor that is inside the membrane. In this way, very flexible loops outside the membrane cannot distort the analysis result. We can manually construct a selection string in MDAnalysis format or load the selections from a file. We call this file ```mor_tm.txt``` and generate it on the fly so we can demonstrate the loader function. We use selections based on the definitions of transmembrane helices in the [GPCRdb](https://gpcrdb.org/protein/oprm_human/).

In [ ]:
! echo "76 98\n105 133\n138 173\n182 208\n226 264\n270 308\n315 354" > mor_tm.txt
! cat mor_tm.txt

In [ ]:
# Load the selection and generate the strings
sel_string_a = load_selection("mor_tm.txt", sel_base_a+" and ")
print('Selection A:\n', sel_string_a, '\n')
sel_string_b = load_selection("mor_tm.txt", sel_base_b+" and ")
print('Selection B:\n', sel_string_b, '\n')
# Extract the coordinates of the transmembrane region from the trajectory
extract_coordinates(ref_file_a, pdb_file_a, [trj_file_a], out_name_a+"_tm", sel_string_a)
extract_coordinates(ref_file_b, pdb_file_b, [trj_file_b], out_name_b+"_tm", sel_string_b)

### Generalization
If you want to combine data from different simulation conditions, you can use the ```_combined``` version of the extraction function: ```extract_coordinates_combined()```. It takes lists as arguments for the topology files, too. To use the same selection, "multiply" a list of one string, as demonstrated below. For this to work, the two selections need to have the exactly same atoms. 

In [ ]:
extract_coordinates_combined([ref_file_a]*3 + [ref_file_b]*3,
                             trj_file_a + trj_file_b, 
                             [sel_string_a]*3 + [sel_string_b]*3, 
                             'traj/combined_tm.xtc', 
                             start_frame=400)

## Featurization

The analysis is not performed on the coordinates directly but on features derived from these coordinates.
PENSA uses the featurization provided by PyEMMA, so far including:
 - backbone torsions: ```'bb-torsions'```, 
 - backbone C-alpha distances: ```'bb-distances'```, and 
 - sidechain torsions: ```'sc-torsions'```.

You can combine these with any other function implemented in PyEMMA, even if it is not included in PENSA.

In case the equilibration phase has not been already excluded during preprocessing, we can exclude it here by setting the start frame to a value greater than 0.

In [ ]:
start_frame = 400

The function ```get_features``` loads the names of the features and their values separately 

In [ ]:
sim_a_rec = get_features("traj/condition-a_receptor.gro", 
                         "traj/condition-a_receptor.xtc", 
                         start_frame)
sim_a_rec_feat, sim_a_rec_data = sim_a_rec

In [ ]:
sim_b_rec = get_features("traj/condition-b_receptor.gro",
                         "traj/condition-b_receptor.xtc", 
                         start_frame)
sim_b_rec_feat, sim_b_rec_data = sim_b_rec

Having a look at the shape of the loaded data, we see that the first dimension is the number of frames. The second dimension is the number of features. It must be the same for both simulations.

In [ ]:
for k in sim_a_rec_data.keys(): 
    print(k, sim_a_rec_data[k].shape)

In [ ]:
for k in sim_b_rec_data.keys(): 
    print(k, sim_b_rec_data[k].shape)

Now do the same only for the transmembrane region.

In [ ]:
sim_a_tmr = get_features("traj/condition-a_tm.gro", 
                         "traj/condition-a_tm.xtc", 
                         start_frame)
sim_b_tmr = get_features("traj/condition-b_tm.gro", 
                         "traj/condition-b_tm.xtc", 
                         start_frame)
sim_a_tmr_feat, sim_a_tmr_data = sim_a_tmr
sim_b_tmr_feat, sim_b_tmr_data = sim_b_tmr

## Comparison of Structural Ensembles

Here we compare the two ensembles using measures for the relative entropy.

You can as well calculate the Kolmogorov-Smirnov metric and the corresponding p value using the function ```kolmogorov_smirnov_analysis()```. 

Another possibility is to compare only the means and standard deviations of the distributions using ```mean_difference_analysis()```.

### Backbone Torsions

We start with the backbone torsions, which we can select via ```'bb-torsions'```. To do the same analysis on sidechain torsions, replace ```'bb-torsions'``` with ```'sc-torsions'```.

In [ ]:
# Relative Entropy analysis with torsions
relen = relative_entropy_analysis(sim_a_rec_feat['bb-torsions'], 
                                  sim_b_rec_feat['bb-torsions'], 
                                  sim_a_rec_data['bb-torsions'], 
                                  sim_b_rec_data['bb-torsions'],
                                  bin_num=10, verbose=False)
names_bbtors, jsd_bbtors, kld_ab_bbtors, kld_ba_bbtors = relen 

The above function also returns the Kullback-Leibler divergences of A with respect to B and vice versa.

To find out where the ensembles differ the most, let's print out the most different features and the corresponding value.

In [ ]:
# Print the features with the 12 highest values
sf = sort_features(names_bbtors, jsd_bbtors)
for f in sf[:12]: print(f[0], f[1])

To get an overview of how strongly the ensembles differ in which region, we can plot the maximum deviation of the features related to a certain residue.

In [ ]:
# Plot the maximum Jensen-Shannon distance per residue as "B factor" in a PDB file
ref_filename = "traj/condition-a_receptor.gro"
out_filename = "receptor_bbtors-deviations_tremd"
vis = residue_visualization(names_bbtors, jsd_bbtors, ref_filename, 
                            "plots/"+out_filename+"_jsd.pdf", 
                            "vispdb/"+out_filename+"_jsd.pdb",
                            y_label='max. JS dist. of BB torsions')


In [ ]:
# Save the corresponding data
np.savetxt('results/'+out_filename+'_relen.csv', 
           np.array(relen).T, fmt='%s', delimiter=',', 
           header='Name, JSD(A,B), KLD(A,B), KLD(B,A)')
np.savetxt('results/'+out_filename+'_jsd.csv', 
           np.array(vis).T, fmt='%s', delimiter=',', 
           header='Residue, max. JSD(A,B)')

### Backbone C-alpha Distances

Another common representation for the overall structure of a protein are the distances between the C-alpha atoms. We can perform the same analysis on them.

In [ ]:
# Relative entropy analysis for C-alpha distances
relen = relative_entropy_analysis(sim_a_rec_feat['bb-distances'], 
                                  sim_b_rec_feat['bb-distances'], 
                                  sim_a_rec_data['bb-distances'], 
                                  sim_b_rec_data['bb-distances'],
                                  bin_num=10, verbose=False)
names_bbdist, jsd_bbdist, kld_ab_bbdist, kld_ba_bbdist = relen 

In [ ]:
# Print the features with the 12 highest values
sf = sort_features(names_bbdist, jsd_bbdist)
for f in sf[:12]: print(f[0], f[1])

To visualize distances, we need a two-dimensional representation with the residues on each axis. 
We color each field with the value of the Jensen-Shannon distance (but could as well use Kullback-Leibler divergence, Kolmogorov-Smirnov statistic etc. instead).

In [ ]:
# Visualize the deviations in a matrix plot
matrix = distances_visualization(names_bbdist, jsd_bbdist, 
                                 "plots/receptor_jsd-bbdist.pdf",
                                 vmin = 0.0, vmax = 1.0)

## Principal Component Analysis

Here we show how to calculate the principal components in the space of backbone torsions. It is also common to calculate principal components in the space of backbone distances. For the latter, again just change ```'bb-torsions'``` to ```'bb-distances'```. As mentioned above, we only consider the transmembrane region here, so flexible loops outside the membrane do not distort the more important slow motions in the receptor core.

#### Combined PCA

In the spirit of comparing two simulations, we calculate the principal components of their joint ensemble of structures.

In [ ]:
# Combine the data of the different simulations
combined_data_tors = np.concatenate([sim_a_tmr_data['bb-torsions'],sim_b_tmr_data['bb-torsions']],0)

We can now calculate the principal components of this combined dataset. The corresponding function returns a PyEMMA PCA object, so you can combine it with all functionality in PyEMMA to perform more advanced or specialized analysis.

In [ ]:
pca_combined = calculate_pca(combined_data_tors)

To find out how relevant each PC is, let's have a look at their eigenvalues.

In [ ]:
pca_eigenvalues_plot(pca_combined, num=12, plot_file='plots/combined_tmr_eigenvalues.pdf')

Let us now have a look at the most relevant features of the first three principal components. 
Here, we define a feature as important if its correlation with the respective PC is above a threshold of 0.4.
The function also plots the correlation analysis for each PC.

In [ ]:
pca_features(pca_combined,sim_a_tmr_feat['bb-torsions'], 3, 0.4)

Now we can compare how the frames of each ensemble are distributed along the principal components.

In [ ]:
compare_projections(sim_a_tmr_data['bb-torsions'],
                    sim_b_tmr_data['bb-torsions'],
                    pca_combined,
                    label_a='A', 
                    label_b='B')

To get a better glimpse on what the Principal components look like, we would like to visualize them. 
For that purpose, let us sort the structures from the trajectories along the principal components instead of along simulation time.
We can then look at the resulting PC trajectories with a molecular visualization program like VMD.

The trajectory to be sorted does not have to be the same subsystem from which we calcualted the PCA. Here, we are going to write frames with the entire receptor, sorted by the PCs of the transmembrane region.

In [ ]:
sort_trajs_along_common_pc(sim_a_tmr_data['bb-torsions'],
                           sim_b_tmr_data['bb-torsions'],
                           start_frame,
                           "traj/condition-a_receptor.gro",
                           "traj/condition-b_receptor.gro",
                           "traj/condition-a_receptor.xtc",
                           "traj/condition-b_receptor.xtc",
                           "pca/receptor_by_tmr",
                           num_pc=3)

The above function deals with the special case of two input trajectories. We also provide the functions for a single one (see below). You use these to calculate PCA for any number of combined simulations and then sort the single or combined simulations.

#### Single simulation

Here are the major steps of a PCA demonstrated for a single simulation.

In [ ]:
pca_a = calculate_pca(sim_a_tmr_data['bb-torsions'])

In [ ]:
pca_features(pca_a, sim_a_tmr_feat['bb-torsions'], 3, 0.4)

In [ ]:
sort_traj_along_pc(sim_a_tmr_data['bb-torsions'], pca_a, start_frame, 
                   "traj/condition-a_receptor.gro", 
                   "traj/condition-a_receptor.xtc", 
                   "pca/condition-a_receptor_by_tmr", num_pc=3)

## Clustering

To identify important states of an ensemble, we can use clustering algorithms. Here we show how to cluster a combined ensemble from two simulations into two clusters using k-means clustering. The plot shows how many frames from which simulation were sorted in which cluster.

In [ ]:
cc = obtain_combined_clusters(sim_a_tmr_data['bb-torsions'],sim_b_tmr_data['bb-torsions'],
                              label_a='A', label_b='B', start_frame=0,
                              algorithm='kmeans', max_iter=100, num_clusters=3, min_dist=12,
                              saveas='plots/combined_clust_bbtors.pdf')
cidx, cond, oidx, wss, centroids = cc

In [ ]:
np.savetxt('results/combined-cluster-indices.csv', 
           np.array([cidx, cond, oidx], dtype=int).T,
           delimiter=',', fmt='%i',
           header='Cluster, Condition, Index within condition')

We can sort the frames from each ensemble into these clusters, writing them as separate trajectory files. As with pricipal components, we can look at them using VMD.

In [ ]:
name = "condition-a_tm"
write_cluster_traj(cidx[cond==0], "traj/"+name+".gro","traj/"+name+".xtc", 
                   "clusters/"+"combined_clust_bbtors_"+name, start_frame )

name = "condition-b_tm"
write_cluster_traj(cidx[cond==1], "traj/"+name+".gro","traj/"+name+".xtc", 
                   "clusters/"+"combined_clust_bbtors_"+name, start_frame )

A common method to obtain the optimal number of clusters is the elbow plot. We plot the with-in-sum-of-squares (WSS) for a few repetitions for an increasing number of clusters.  Then we look for the "elbow" in the resulting plot. Unfortunately, sometimes there is no clear result though.

In [ ]:
wss_avg, wss_std = wss_over_number_of_combined_clusters(sim_a_tmr_data['bb-torsions'], 
                                                        sim_b_tmr_data['bb-torsions'],
                                                        label_a='A', label_b='B', 
                                                        start_frame=start_frame,
                                                        algorithm='kmeans', 
                                                        max_iter=100, num_repeats = 5, 
                                                        max_num_clusters = 12, 
                                                        plot_file = None)

Of course, we can also cluster a single simulation

In [ ]:
_ci, _wss, _centroids = obtain_clusters( sim_a_tmr_data['bb-torsions'], num_clusters=5 )
name = "condition-a_tm"
write_cluster_traj( _ci, "traj/"+name+".gro","traj/"+name+".xtc", 
                   "clusters/"+"clust_bbtors_"+name, start_frame )

In [ ]:
wss_avg, wss_std = wss_over_number_of_clusters(sim_a_tmr_data['bb-torsions'],
                                               algorithm='kmeans', 
                                               max_iter=100, num_repeats = 5, 
                                               max_num_clusters = 12, 
                                               plot_file = None)